![stack](images/kim.jpg)

# Polar Data Insights and Search Analytics for the Deep and Scientific Web 

The pitch - 

"Our preliminary work in this area has shown that the unstructured textual data, when combined with structured scientific information can inform answers to grand challenge problems such as identifying ice sheet breakage/melt over decadal time spans; bird migration around Greenland, oil spills and natural disasters, sea ice decline and its relation to natural disasters, and other critical questions for the Polar community derived from the President’s National Strategy for the Arctic Region."


## Polar Data Insights (PDI for short)

![stack](images/stack.png)

In [1]:
from nltk.tokenize import RegexpTokenizer
from nltk.stem.wordnet import WordNetLemmatizer
from gensim import corpora, models
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
import json

import urllib.request

solr_root = 'http://integration.polar-deep-vm.apps.int.nsidc.org:8983/solr/rda/select?fl=url,content,type&indent=on&'
solr_query = 'q=url:"case-statement"&rows=200&start=1&wt=json'
solr_url = solr_root + solr_query
req = urllib.request.Request(solr_url)
# parsing response
r = urllib.request.urlopen(req).read()
json_response = json.loads(r.decode('utf-8'))
solr_documents = json_response['response']['docs']
print("Processing {0} documents. \n".format(len(solr_documents)))

Processing 170 documents. 



* *Now we tokenize each document and remove stop words and apply stemming (Wordnet lemmatizer)*


In [2]:
## we need a tokenizer
tokenizer = RegexpTokenizer(r'\w+')
## we need stemer
stemmer = WordNetLemmatizer()
## our custom stop words
my_stop_words = {
                    'http', 'www', 'edu', 'org', 'com', 'rda', 'data', 'researcher', 'event', 'service',
                    'group', 'research', 'community', 'use', 'work', 'member', 'case', 'working', 'science',
                    'meeting', 'organisational', 'news', 'plenary', 'recommendation', 'project', 'standard',
                    'statement', 'school', 'university', 'membership', 'output', '2017', 'brokering',
                    'stakeholder', 'repository', 'user', 'citation', 'chair', 'framework', 'information',
                    'metadata', 'content', 'sharing', 'pid'
                }
stop_words = my_stop_words.union(ENGLISH_STOP_WORDS)

# document list will contain our corpus after cleaning it.
document_list = []
# pairs is a list of the urls and the size of their content
pairs = []
# just the documents urls
urls = []

for item in solr_documents:
    # If we apply NER it should be the first step.
    # We tokenize words and lower case them(for now)
    tokens = tokenizer.tokenize((item['content'][0]).lower())
    # We lematize (stemming)
    stemmed_tokens = [stemmer.lemmatize(i) for i in tokens]
    # If the token is not in our stop words and the length is >2 and <20 we add it to the cleaned document
    document = [i for i in stemmed_tokens if i not in stop_words and (len(i) > 2 and len(i) < 25)]
    # To debug uncomment the next line
    # print("{0}\n Document size before stop words: {1}, after: {2} ".format(item['url'],len(stemmed_tokens),len(document)))
    document_list.append(document)
    pairs.append((item['url'],len(document)))
    urls.append(item['url'])

* Building the LDA model using Gensim a library for topic modeling, the output is a list of topics present in our corpus.

In [3]:
num_passes = 10
num_topics = 20
words_per_topic = 6

dictionary = corpora.Dictionary(document_list)
corpus = [dictionary.doc2bow(text) for text in document_list]
lda_model = models.ldamodel.LdaModel(corpus, num_topics=num_topics, id2word = dictionary, passes=num_passes)
topics = lda_model.print_topics(num_topics=num_topics, num_words=words_per_topic)
# Now let's print the topics found
for topic in topics:
    print(topic)

(0, '0.012*"indigenous" + 0.009*"international" + 0.008*"sov" + 0.008*"policy" + 0.006*"creating" + 0.006*"network"')
(1, '0.013*"array" + 0.011*"database" + 0.008*"chemistry" + 0.006*"library" + 0.005*"chemical" + 0.005*"domain"')
(2, '0.010*"humanity" + 0.009*"empirical" + 0.007*"wheat" + 0.007*"practice" + 0.006*"developing" + 0.006*"provide"')
(3, '0.009*"archive" + 0.009*"transportation" + 0.009*"record" + 0.008*"professional" + 0.007*"geospatial" + 0.006*"usa"')
(4, '0.063*"cost" + 0.039*"recovery" + 0.025*"model" + 0.024*"centre" + 0.013*"study" + 0.010*"policy"')
(5, '0.013*"software" + 0.010*"code" + 0.010*"source" + 0.009*"outcome" + 0.008*"creating" + 0.008*"deliverable"')
(6, '0.028*"rice" + 0.013*"national" + 0.011*"interoperability" + 0.006*"international" + 0.006*"legal" + 0.006*"codata"')
(7, '0.020*"preservation" + 0.012*"tool" + 0.007*"policy" + 0.007*"career" + 0.007*"usa" + 0.007*"early"')
(8, '0.009*"type" + 0.006*"creating" + 0.006*"past" + 0.005*"plenaries" + 0.0

* The following code builds a list of all the documents that belong to a particular topic and their calculated probabilities

In [4]:
import ipywidgets as widgets

minum_likelihood = 0.1 # 10%
total_documents = 169

# We create a list for each topic containing the terms and an empty document list
docs_in_topics = [{'topic':t[0],'terms': t[1], 'documents': []} for t in topics]
# Add each document to their predicted topics if the probability is above the minimum_likelihood
for i in range(total_documents):
    doc_prob = lda_model.get_document_topics(bow=corpus[i],minimum_probability=minum_likelihood)
    document_url = urls[i]
    # Each document could contain more than one topic, we traverse them and add the url to n topics
    for prob in doc_prob:
        topic_index = prob[0]
        topic_probability = prob[1]
        docs_in_topics[topic_index]['documents'].append((document_url,topic_probability))

* Now we can select a topic and then we'll print all the documents for it.

In [5]:
topic_of_interest = 11 # the topic index

def getkey(doc):
    return doc[1]

print("Documents in Topic {0} ({1})".format(topic_of_interest,docs_in_topics[topic_of_interest]['terms']))
for doc in sorted(docs_in_topics[topic_of_interest]['documents'],key=getkey):
    print(" Document: {0} \n - Probability:{1}".format(doc[0],doc[1]))

Documents in Topic 11 (0.000*"national" + 0.000*"publication" + 0.000*"new" + 0.000*"practice" + 0.000*"infrastructure" + 0.000*"creating")


* After we execute the next cell we can pick any of the 200 URLs and see what LDA predicted for the selected URL

In [6]:
url_list = widgets.Dropdown(
    options=urls,
    description='URL:',
    disabled=False,)
display(url_list)

Dropdown(description='URL:', options=('https://rd-alliance.org/group/data-development/case-statement/data-development-case-statement.html', 'https://rd-alliance.org/group/data-fabric-ig/case-statement/data-fabric-ig-case-statement.html', 'https://rd-alliance.org/group/data-versioning-wg/case-statement/data-versioning-wg-case-statement', 'https://rd-alliance.org/group/working-group-international-materials-resource-registries/case-statement/case-statement-rda', 'https://rd-alliance.org/group/provenance-patterns-wg/case-statement/working-group-provenance-patterns-case-statement', 'https://rd-alliance.org/sites/default/files/case_statement/Case_Statement_IG_DataforDevelopment_July2014.docx', 'https://rd-alliance.org/group/brokering-governance/case-statement/rda-brokering-governance-case-statement.html', 'https://rd-alliance.org/group/data-foundations-and-terminology-ig/case-statement/case-statement.html', 'https://rd-alliance.org/group/pid-collections-wg/case-statement/pid-collections-wg-case-statement.html', 'https://rd-alliance.org/group/reproducibility-ig/case-statement/reproducibility-ig-case-statement.html', 'https://rd-alliance.org/sites/default/files/case_statement/PIDKernelInformationWG-case-statement-170516-for-TAB.pdf', 'https://rd-alliance.org/sites/default/files/case_statement/rda-rprd-case-statement.pdf', 'https://rd-alliance.org/sites/default/files/case_statement/RDA_IG_case_statement.pdf', 'https://rd-alliance.org/group/data-citation-wg/case-statement/wg-data-citation-making-data-citable-case-statement.html', 'https://rd-alliance.org/group/dmp-common-standards-wg/case-statement/rda-wg-dmp-common-standards-case-statement', 'https://rd-alliance.org/group/international-materials-resource-registries-wg/case-statement/case-statement-rda-working-group', 'https://rd-alliance.org/group/qos-datalc-definitions-wg/case-statement/qos-datalc-definitions-wg-case-statement', 'https://rd-alliance.org/group/rdawds-publishing-data-services-wg/case-statement/rdawds-publishing-services-wg-case-statement', 'https://rd-alliance.org/group/elixir-bridging-force-ig/case-statement/elixir-bridging-force-ig-case-statement.html', 'https://rd-alliance.org/group/wheat-data-interoperability-wg/case-statement/wheat-data-interoperability-wg-case-statement', 'https://rd-alliance.org/group/libraries-research-data/case-statement/libraries-research-data-ig-case-statement.html', 'https://rd-alliance.org/group/pid-kernel-information-wg/case-statement/pid-kernel-information-wg-case-statement', 'https://rd-alliance.org/sites/default/files/case_statement/case-statement-qos-datalc-definitions_v30032017.pdf', 'https://rd-alliance.org/group/research-data-provenance/case-statement/research-data-provenance-ig-case-statement-2017', 'https://rd-alliance.org/group/urban-quality-life-indicators/case-statement/urban-quality-life-indicators-wg-case-statement', 'https://rd-alliance.org/group/weather-climate-and-air-quality/case-statement/weather-climate-and-air-quality-case-statement', 'https://rd-alliance.org/group/farm-data-sharing-ofds-wg/case-statement/draft-case-statement-create-working-group-entitled-%E2%80%9C', 'https://www.rd-alliance.org/revised-weather-climate-and-air-quality-ig-case-statement', 'https://www.rd-alliance.org/sites/default/files/case_statement/Agrisemantics_WG_CaseStatement_0.pdf', 'https://www.rd-alliance.org/sites/default/files/case_statement/AttributionWGCaseStatement.pdf', 'https://www.rd-alliance.org/sites/default/files/case_statement/AttributionWGCaseStatement_revised.pdf', 'https://www.rd-alliance.org/sites/default/files/case_statement/ADMP-IG-Case_v3.docx', 'https://www.rd-alliance.org/sites/default/files/case_statement/DraftWGCaseStatementforPublishingDataServices2WG_20161007_0.docx', 'https://www.rd-alliance.org/sites/default/files/case_statement/RDACODATA_SummerSchoolsForLowMiddleIncomeCountries_IGCharter_20170911.docx', 'https://rd-alliance.org/group/agrisemantics-wg/case-statement/agrisemantics-working-group', 'https://rd-alliance.o

In [7]:
# Each time we pick a new URL we should execute this cell
current_url = str(url_list.value)
print("LDA predictions for:\n" + current_url)
for topic in docs_in_topics:
    for doc in topic['documents']:
        if (current_url == doc[0]):
            print("Topic {0} Probability {1} \n - Terms: {2}".format(docs_in_topics.index(topic), doc[1],topic['terms']))

LDA predictions for:
https://rd-alliance.org/group/data-development/case-statement/data-development-case-statement.html
Topic 10 Probability 0.998359203338623 
 - Terms: 0.010*"linguistics" + 0.009*"linguistic" + 0.008*"ldig" + 0.008*"health" + 0.007*"development" + 0.006*"language"


In [8]:
import pyLDAvis.gensim
import warnings
pyLDAvis.enable_notebook()
warnings.filterwarnings('ignore')

/opt/miniconda/lib/python3.5/site-packages/funcy-1.7.2-py3.5.egg/funcy/decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
/opt/miniconda/lib/python3.5/site-packages/funcy-1.7.2-py3.5.egg/funcy/decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
/opt/miniconda/lib/python3.5/site-packages/funcy-1.7.2-py3.5.egg/funcy/decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
/opt/miniconda/lib/python3.5/site-packages/funcy-1.7.2-py3.5.egg/funcy/decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
/opt/miniconda/lib/python3.5/site-packages/funcy-1.7.2-py3.5.egg/funcy/decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
/opt/miniconda/lib/python3.5/site-packages/funcy-1.7.2-py3.5.egg/funcy/decorators.py:56: DeprecationWarning: inspec

In [9]:
pyLDAvis.gensim.prepare(corpus=corpus, topic_model=lda_model, dictionary=dictionary, sort_topics=False)

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
0       7.607866        1       1 -0.046473 -0.079644
1       2.377963        1       2  0.030190  0.007940
2       4.027495        1       3 -0.069191  0.030099
3       3.321760        1       4  0.046898 -0.084829
4       0.736220        1       5  0.145895  0.184719
5       1.041958        1       6  0.048412 -0.039197
6       6.442435        1       7 -0.082571  0.007814
7       2.681479        1       8  0.010669 -0.030109
8      17.747622        1       9 -0.017782 -0.103107
9       6.197826        1      10 -0.107287  0.060644
10      4.074594        1      11  0.002049 -0.048166
11      0.005065        1      12  0.189526  0.079081
12      8.877875        1      13 -0.078549 -0.023378
13      9.193937        1      14 -0.100702  0.062256
14      1.754004        1      15  0.009717  0.120098
15      4.369071        1      16 -0.046464 -0.005233
16      5.583203        1      17 -0.073297  0.024372
17      2.847305        1      18  0.087082 -0.160973
18      0.718786        1      19  0.175987 -0.045489
19     10.393524        1      20 -0.124109  0.043103, topic_info=      Category        Freq              Term       Total  loglift  logprob
term                                                                      
1179   Default  458.000000             model  458.000000  30.0000  30.0000
478    Default  305.000000              rice  305.000000  29.0000  29.0000
9461   Default  150.000000              cost  150.000000  28.0000  28.0000
4099   Default  245.000000           privacy  245.000000  27.0000  27.0000
10308  Default  165.000000            centre  165.000000  26.0000  26.0000
5948   Default  519.000000          creating  519.000000  25.0000  25.0000
12138  Default  166.000000        indigenous  166.000000  24.0000  24.0000
9972   Default  103.000000           storage  103.000000  23.0000  23.0000
10740  Default  480.000000            policy  480.000000  22.0000  22.0000
846    Default  448.000000               set  448.000000  21.0000  21.0000
7839   Default  342.000000        collection  342.000000  20.0000  20.0000
10798  Default  349.000000           related  349.000000  19.0000  19.0000
2425   Default  467.000000              past  467.000000  18.0000  18.0000
7166   Default  156.000000      preservation  156.000000  17.0000  17.0000
11870  Default  612.000000          national  612.000000  16.0000  16.0000
8547   Default  195.000000          software  195.000000  15.0000  15.0000
932    Default  408.000000            report  408.000000  14.0000  14.0000
9741   Default  268.000000          document  268.000000  13.0000  13.0000
8355   Default  162.000000             study  162.000000  12.0000  12.0000
10416  Default  456.000000              plan  456.000000  11.0000  11.0000
9120   Default  281.000000        vocabulary  281.000000  10.0000  10.0000
11575  Default  264.000000        publishing  264.000000   9.0000   9.0000
11480  Default  210.000000          database  210.000000   8.0000   8.0000
7413   Default  265.000000       deliverable  265.000000   7.0000   7.0000
2090   Default  447.000000       publication  447.000000   6.0000   6.0000
6616   Default  182.000000            survey  182.000000   5.0000   5.0000
2030   Default  343.000000             issue  343.000000   4.0000   4.0000
3229   Default  391.000000           outcome  391.000000   3.0000   3.0000
5797   Default  111.000000         mediation  111.000000   2.0000   2.0000
4793   Default  225.000000         component  225.000000   1.0000   1.0000
...        ...         ...               ...         ...      ...      ...
579    Topic20   47.051785              inra   69.474838   1.8743  -5.9141
1769   Topic20   65.021606         semantics  102.528015   1.8086  -5.5906
643    Topic20   43.152908             asset   63.236694   1.8819  -6.0006
4636   Topic20   25.521572            stream   34.203537   1.9712  -6

### References and Links



> L. A. Lopez, R. Duerr and S. J. S. Khalsa, "Optimizing apache nutch for domain specific crawling at large scale," 2015 IEEE International Conference on Big Data (Big Data), Santa Clara, CA, 2015, pp. 1967-1971.
doi: 10.1109/BigData.2015.7363976

-

> 